# **Jane Street Market Prediction: A simple EDA and prediction**


**"Machine learning (ML) at Jane Street begins, unsurprisingly, with data. We collect and store around 2.3TB of market data every day. Hidden in those petabytes of data are the relationships and statistical regularities which inform the models inside our strategies. But it’s not just awesome models. ML work in a production environment like Jane Street’s involves many interconnected pieces." -- Jane Street Tech Blog "Real world machine learning".**

# **1.Import**

In [ ]:
# numpy
import numpy as np

# pandas stuff
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# plotting stuff
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
colorMap = sns.light_palette("blue", as_cmap=True)
#plt.rcParams.update({'font.size': 12})


# install dabl
!pip install dabl > /dev/null
import dabl
# install datatable
!pip install datatable > /dev/null
import datatable as dt

# misc
import missingno as msno

# system
import warnings
warnings.filterwarnings('ignore')
# for the image import
import os
from IPython.display import Image
# garbage collector to keep RAM in check
import gc

# **2.The train.csv file is big**

In [ ]:
!wc -l ../input/jane-street-market-prediction/train.csv

**We can see that it has a total of 2,390,492 rows. I recommend reading this magnificent Tutorial on reading large datasets by Vopani.

I have used pandas to load in the train.csv and it took almost 2 minutes. To speed things up here I shall use datatable:**

In [ ]:
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
train_data = train_data_datatable.to_pandas()

# **3.Deal with feather---resp**

**There are a total of 500 days of data in train.csv (i.e. two years of trading data). Let us take a look at the cumulative values of resp over time**

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['resp']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_ylabel ("Cumulative resp", fontsize=18);
balance.plot(lw=3);
del balance
gc.collect();

**as well as four time horizons**

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['resp']).cumsum()
resp_1= pd.Series(train_data['resp_1']).cumsum()
resp_2= pd.Series(train_data['resp_2']).cumsum()
resp_3= pd.Series(train_data['resp_3']).cumsum()
resp_4= pd.Series(train_data['resp_4']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
balance.plot(lw=3)
resp_1.plot(lw=3)
resp_2.plot(lw=3)
resp_3.plot(lw=3)
resp_4.plot(lw=3)
plt.legend(loc="upper left");
del resp_1
del resp_2
del resp_3
del resp_4
gc.collect();

**Let us now plot a histogram of all of the resp values (here only shown for values between -0.05 and 0.05)**

In [ ]:
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['resp'], 
             bins=3000, 
             kde_kws={"clip":(-0.05,0.05)}, 
             hist_kws={"range":(-0.05,0.05)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the resp values", size=14)
plt.show();
gc.collect();

**This distribution has very long tails**

In [ ]:
min_resp = train_data['resp'].min()
print('The minimum value for resp is: %.5f' % min_resp)
max_resp = train_data['resp'].max()
print('The maximum value for resp is:  %.5f' % max_resp)

**Let us also calculate the skew and kurtosis of this distribution:**

In [ ]:
print("Skew of resp is:      %.2f" %train_data['resp'].skew() )
print("Kurtosis of resp is: %.2f"  %train_data['resp'].kurtosis() )

**Finally, let us fit a Cauchy distribution to this data**

In [ ]:
from scipy.optimize import curve_fit
# the values
x = list(range(len(values)))
x = [((i)-1500)/30000 for i in x]
y = values

def Lorentzian(x, x0, gamma, A):
    return A * gamma**2/(gamma**2+( x - x0 )**2)

# seed guess
initial_guess=(0, 0.001, 3000)

# the fit
parameters,covariance=curve_fit(Lorentzian,x,y,initial_guess)
sigma=np.sqrt(np.diag(covariance))

# and plot
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['resp'], 
             bins=3000, 
             kde_kws={"clip":(-0.05,0.05)}, 
             hist_kws={"range":(-0.05,0.05)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
#norm = plt.Normalize(values.min(), values.max())
#colors = plt.cm.jet(norm(values))
#for rec, col in zip(ax.patches, colors):
#    rec.set_color(col)
plt.xlabel("Histogram of the resp values", size=14)
plt.plot(x,Lorentzian(x,*parameters),'--',color='black',lw=3)
plt.show();
del values
gc.collect();

# **4.Weight**

Each trade has an associated weight and resp, which together represents a return on the trade. Trades with weight = 0 were intentionally included in the dataset for completeness, although such trades will not contribute towards the scoring evaluation.

In [ ]:
percent_zeros = (100/train_data.shape[0])*((train_data.weight.values == 0).sum())
print('Percentage of zero weights is: %i' % percent_zeros +"%")

**An now to find the maximum weight used**

In [ ]:
max_weight = train_data['weight'].max()
print('The maximum weight was: %.2f' % max_weight)

**which occured on day 446**

In [ ]:
train_data[train_data['weight']==train_data['weight'].max()]

**Let us take a look at a histogram of the non-zero weights**

In [ ]:
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['weight'], 
             bins=1400, 
             kde_kws={"clip":(0.001,1.4)}, 
             hist_kws={"range":(0.001,1.4)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of non-zero weights", size=14)
plt.show();
del values
gc.collect();

# **5.Cumulative return**

Let us take a look at the cumulative daily return over time, which is given by weight multiplied by the value of resp

In [ ]:
train_data['weight_resp']   = train_data['weight']*train_data['resp']
train_data['weight_resp_1'] = train_data['weight']*train_data['resp_1']
train_data['weight_resp_2'] = train_data['weight']*train_data['resp_2']
train_data['weight_resp_3'] = train_data['weight']*train_data['resp_3']
train_data['weight_resp_4'] = train_data['weight']*train_data['resp_4']

fig, ax = plt.subplots(figsize=(15, 5))
resp    = pd.Series(1+(train_data.groupby('date')['weight_resp'].mean())).cumprod()
resp_1  = pd.Series(1+(train_data.groupby('date')['weight_resp_1'].mean())).cumprod()
resp_2  = pd.Series(1+(train_data.groupby('date')['weight_resp_2'].mean())).cumprod()
resp_3  = pd.Series(1+(train_data.groupby('date')['weight_resp_3'].mean())).cumprod()
resp_4  = pd.Series(1+(train_data.groupby('date')['weight_resp_4'].mean())).cumprod()
ax.set_xlabel ("Day", fontsize=18)
ax.set_title ("Cumulative daily return for resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
resp.plot(lw=3, label='resp x weight')
resp_1.plot(lw=3, label='resp_1 x weight')
resp_2.plot(lw=3, label='resp_2 x weight')
resp_3.plot(lw=3, label='resp_3 x weight')
resp_4.plot(lw=3, label='resp_4 x weight')
# day 85 marker
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
plt.legend(loc="lower left");

We can see that the shortest time horizons, resp_1, resp_2 and resp_3, representing a more conservative strategy, result in the lowest return.

We shall now plot a histogram of the weight multiplied by the value of resp (after removing the 0 weights)

In [ ]:
train_data_no_0 = train_data.query('weight > 0').reset_index(drop = True)
train_data_no_0['wAbsResp'] = train_data_no_0['weight'] * (train_data_no_0['resp'])
#plot
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data_no_0['wAbsResp'], 
             bins=1500, 
             kde_kws={"clip":(-0.02,0.02)}, 
             hist_kws={"range":(-0.02,0.02)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the weights * resp", size=14)
plt.show();

# **6.Time**

Let us plot the number of ts_id per day. Note: I have taken to drawing a vertical dashed line in my plots because I started to wonder did Jane Street modify their trading model around day 85? Thanks to comments on that forum the general consenus seems to be that a change in the market took place around that time (perhaps a mean reverting market changing to a momentum market, or vice versa).

In [ ]:
trades_per_day = train_data.groupby(['date'])['ts_id'].count()
fig, ax = plt.subplots(figsize=(15, 5))
plt.plot(trades_per_day)
ax.set_xlabel ("Day", fontsize=18)
ax.set_title ("Total number of ts_id for each day", fontsize=18)
# day 85 marker
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
plt.show()

If we assume a trading day is 6½ hours long (i.e. 23400 seconds) then

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
plt.plot(23400/trades_per_day)
ax.set_xlabel ("Day", fontsize=18)
ax.set_ylabel ("Av. time between trades (s)", fontsize=18)
ax.set_title ("Average time between trades for each day", fontsize=18)
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax.set_xlim(xmin=0)
ax.set_xlim(xmax=500)
ax.set_ylim(ymin=0)
ax.set_ylim(ymax=12)
plt.show()

Here is a histogram of the number of trades per day (it has been suggested that the number of trades per day is an indication of the volatility that day)

In [ ]:
plt.figure(figsize = (12,4))
# the minimum has been set to 1000 so as not to draw the partial days like day 2 and day 294
# the maximum number of trades per day is 18884
# I have used 125 bins for the 500 days
ax = sns.distplot(trades_per_day, 
             bins=125, 
             kde_kws={"clip":(1000,20000)}, 
             hist_kws={"range":(1000,20000)},
             color='darkcyan', 
             kde=True);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Number of trades per day", size=14)
plt.show();

# **7.The features**

"This dataset contains an anonymized set of features, feature_{0...129}, representing real stock market data."

In [ ]:
train_data['feature_0'].value_counts()

Also, feature_0 is the only feature in the features.csv file that has no True tags.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 4))
feature_0 = pd.Series(train_data['feature_0']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_ylabel ("feature_0 (cumulative)", fontsize=18);
feature_0.plot(lw=3);

It is also very interesting to plot the cumulative resp and return (resp*weight) for feature_0 = +1 and feature_0 = -1 individually (Credit: "An observation about feature_0" by therocket290)

In [ ]:
feature_0_is_plus_one  = train_data.query('feature_0 ==  1').reset_index(drop = True)
feature_0_is_minus_one = train_data.query('feature_0 == -1').reset_index(drop = True)
# the plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
ax1.plot((pd.Series(feature_0_is_plus_one['resp']).cumsum()), lw=3, label='resp')
ax1.plot((pd.Series(feature_0_is_plus_one['resp']*feature_0_is_plus_one['weight']).cumsum()), lw=3, label='return')
ax2.plot((pd.Series(feature_0_is_minus_one['resp']).cumsum()), lw=3, label='resp')
ax2.plot((pd.Series(feature_0_is_minus_one['resp']*feature_0_is_minus_one['weight']).cumsum()), lw=3, label='return')
ax1.set_title ("feature 0 = 1", fontsize=18)
ax2.set_title ("feature 0 = -1", fontsize=18)
ax1.legend(loc="lower left")
ax2.legend(loc="upper left");

del feature_0_is_plus_one
del feature_0_is_minus_one
gc.collect();

**feature_{1...129}**


There seem to be four general 'types' of features, here is a plot of an example of one of each:

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,figsize=(20,10))

ax1.plot((pd.Series(train_data['feature_1']).cumsum()), lw=3, color='red')
ax1.set_title ("Linear", fontsize=22);
ax1.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax1.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax1.set_xlim(xmin=0)
ax1.set_ylabel ("feature_1", fontsize=18);

ax2.plot((pd.Series(train_data['feature_3']).cumsum()), lw=3, color='green')
ax2.set_title ("Noisy", fontsize=22);
ax2.axvline(x=514052, linestyle='--', alpha=0.3, c='red', lw=2)
ax2.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax2.set_xlim(xmin=0)
ax2.set_ylabel ("feature_3", fontsize=18);

ax3.plot((pd.Series(train_data['feature_55']).cumsum()), lw=3, color='darkorange')
ax3.set_title ("Hybryd (Tag 21)", fontsize=22);
ax3.set_xlabel ("Trade", fontsize=18)
ax3.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax3.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax3.set_xlim(xmin=0)
ax3.set_ylabel ("feature_55", fontsize=18);

ax4.plot((pd.Series(train_data['feature_73']).cumsum()), lw=3, color='blue')
ax4.set_title ("Negative", fontsize=22)
ax4.set_xlabel ("Trade", fontsize=18)
ax4.set_ylabel ("feature_73", fontsize=18);
gc.collect();

The Tag 14 set are interesting as they appear to be "stratified"; only adopting discrete values throughout the day (could these be a value of a security?). Here are scatter plots of these three features for days 0, 1 and 3 (Note that I have omitted day 2, which I shall discuss in the missing data section below):

In [ ]:
day_0 = train_data.loc[train_data['date'] == 0]
day_1 = train_data.loc[train_data['date'] == 1]
day_3 = train_data.loc[train_data['date'] == 3]
three_days = pd.concat([day_0, day_1, day_3])
three_days.plot.scatter(x='ts_id', y='feature_41', s=0.5, figsize=(15,3));
three_days.plot.scatter(x='ts_id', y='feature_42', s=0.5, figsize=(15,3));
three_days.plot.scatter(x='ts_id', y='feature_43', s=0.5, figsize=(15,3));
del day_1
del day_3
gc.collect();

These three features also have very interesting lag plots, where we plot the value of the feature at ts_id  (n)  with respect to the next value of the feature, i.e. at ts_id  (n+1) , (here for day 0). Red markers have been placed at (0,0) as a visual aid.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(17, 4))
lag_plot(day_0['feature_41'], lag=1, s=0.5, ax=ax[0])
lag_plot(day_0['feature_42'], lag=1, s=0.5, ax=ax[1])
lag_plot(day_0['feature_43'], lag=1, s=0.5, ax=ax[2])
ax[0].title.set_text('feature_41')
ax[0].set_xlabel("ts_id (n)")
ax[0].set_ylabel("ts_id (n+1)")
ax[1].title.set_text('feature_42')
ax[1].set_xlabel("ts_id (n)")
ax[1].set_ylabel("ts_id (n+1)")
ax[2].title.set_text('feature_43')
ax[2].set_xlabel("ts_id (n)")
ax[2].set_ylabel("ts_id (n+1)")

ax[0].plot(0, 0, 'r.', markersize=15.0)
ax[1].plot(0, 0, 'r.', markersize=15.0)
ax[2].plot(0, 0, 'r.', markersize=15.0);
gc.collect();

# **8.Data prediction and processing results**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Pytorch Resnet part**

In [ ]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import namedtuple
from sklearn.metrics import log_loss, roc_auc_score

from random import choices

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import warnings
warnings.filterwarnings ("ignore")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

DATA_PATH = '../input/jane-street-market-prediction/'

NFOLDS = 5

TRAIN = False
CACHE_PATH = '../input/mlp012003weights'

def save_pickle(dic, save_path):
    with open(save_path, 'wb') as f:
        pickle.dump(dic, f)

def load_pickle(load_path):
    with open(load_path, 'rb') as f:
        message_dict = pickle.load(f)
    return message_dict

feat_cols = [f'feature_{i}' for i in range(130)]

target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']

f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

##### Making features
all_feat_cols = [col for col in feat_cols]
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])

##### Model&Data fnc
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm0 = nn.BatchNorm1d(len(all_feat_cols))
        self.dropout0 = nn.Dropout(0.2)

        dropout_rate = 0.2
        hidden_size = 256
        self.dense1 = nn.Linear(len(all_feat_cols), hidden_size)
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.dense2 = nn.Linear(hidden_size+len(all_feat_cols), hidden_size)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)

        self.dense3 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)

        self.dense4 = nn.Linear(hidden_size+hidden_size, hidden_size)
        self.batch_norm4 = nn.BatchNorm1d(hidden_size)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.dense5 = nn.Linear(hidden_size+hidden_size, len(target_cols))

        self.Relu = nn.ReLU(inplace=True)
        self.PReLU = nn.PReLU()
        self.LeakyReLU = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        self.RReLU = nn.RReLU()

    def forward(self, x):
        x = self.batch_norm0(x)
        x = self.dropout0(x)

        x1 = self.dense1(x)
        x1 = self.batch_norm1(x1)
        x1 = self.LeakyReLU(x1)
        x1 = self.dropout1(x1)

        x = torch.cat([x, x1], 1)

        x2 = self.dense2(x)
        x2 = self.batch_norm2(x2)
        x2 = self.LeakyReLU(x2)
        x2 = self.dropout2(x2)

        x = torch.cat([x1, x2], 1)

        x3 = self.dense3(x)
        x3 = self.batch_norm3(x3)
        x3 = self.LeakyReLU(x3)
        x3 = self.dropout3(x3)

        x = torch.cat([x2, x3], 1)

        x4 = self.dense4(x)
        x4 = self.batch_norm4(x4)
        x4 = self.LeakyReLU(x4)
        x4 = self.dropout4(x4)

        x = torch.cat([x3, x4], 1)

        x = self.dense5(x)

        return x

if True:
    device = torch.device("cpu")
    
    model_list = []
    tmp = np.zeros(len(feat_cols))
    for _fold in range(NFOLDS):
        torch.cuda.empty_cache()
        model = Model()
        model.to(device)
        model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
        model.load_state_dict(torch.load(model_weights, map_location=torch.device('cpu')))
        model.eval()
        model_list.append(model)

# **Tensorflow part**

In [ ]:
!ls ../input/jane-street-with-keras-nn-overfit

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

SEED = 1111

np.random.seed(SEED)

# fit
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates)(x)
    for i in range(3):
        x = tf.keras.layers.Dense(hidden_units)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates)(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

epochs = 200
batch_size = 4096
hidden_units = 160
dropout_rates = 0.2
label_smoothing = 1e-2
learning_rate = 1e-3

tf.keras.backend.clear_session()
tf.random.set_seed(SEED)
clf = create_mlp(
    len(feat_cols), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
    )
clf.load_weights('../input/jane-street-with-keras-nn-overfit/model.h5')

tf_models = [clf]

# **EMbeddings NN**

In [ ]:
N_FEAT_TAGS = 29
DEVICE = device
N_FEATURES = 130
THREE_HIDDEN_LAYERS = [400, 400, 400]

class FFN (nn.Module):
    
    def __init__(self, inputCount=130, outputCount=5, hiddenLayerCounts=[150, 150, 150], 
                 drop_prob=0.2, nonlin=nn.SiLU (), isOpAct=False):
        
        super(FFN, self).__init__()
        
        self.nonlin     = nonlin
        self.dropout    = nn.Dropout (drop_prob)
        self.batchnorm0 = nn.BatchNorm1d (inputCount)
        self.dense1     = nn.Linear (inputCount, hiddenLayerCounts[0])
        self.batchnorm1 = nn.BatchNorm1d (hiddenLayerCounts[0])
        self.dense2     = nn.Linear(hiddenLayerCounts[0], hiddenLayerCounts[1])
        self.batchnorm2 = nn.BatchNorm1d (hiddenLayerCounts[1])
        self.dense3     = nn.Linear(hiddenLayerCounts[1], hiddenLayerCounts[2])
        self.batchnorm3 = nn.BatchNorm1d (hiddenLayerCounts[2])        
        self.outDense   = None
        if outputCount > 0:
            self.outDense   = nn.Linear (hiddenLayerCounts[-1], outputCount)
        self.outActivtn = None
        if isOpAct:
            if outputCount == 1 or outputCount == 2:
                self.outActivtn = nn.Sigmoid ()
            elif outputCount > 0:
                self.outActivtn = nn.Softmax (dim=-1)
        return

    def forward (self, X):
        
        # X = self.dropout (self.batchnorm0 (X))
        X = self.batchnorm0 (X)
        X = self.dropout (self.nonlin (self.batchnorm1 (self.dense1 (X))))
        X = self.dropout (self.nonlin (self.batchnorm2 (self.dense2 (X))))
        X = self.dropout (self.nonlin (self.batchnorm3 (self.dense3 (X))))
        if self.outDense:
            X = self.outDense (X)
        if self.outActivtn:
            X = self.outActivtn (X)
        return X
    
    
class Emb_NN_Model (nn.Module):
    
    def __init__(self, three_hidden_layers=THREE_HIDDEN_LAYERS, embed_dim=(N_FEAT_TAGS),
                 csv_file='../input/jane-street-market-prediction/features.csv'):
        
        super (Emb_NN_Model, self).__init__()
        global N_FEAT_TAGS
        N_FEAT_TAGS = 29
        dtype = {'tag_0' : 'int8'}
        for i in range (1, 29):
            k = 'tag_' + str (i)
            dtype[k] = 'int8'
        t_df = pd.read_csv (csv_file, usecols=range (1,N_FEAT_TAGS+1), dtype=dtype)
        t_df['tag_29'] = np.array ([1] + ([0] * (t_df.shape[0]-1)) ).astype ('int8')
        self.features_tag_matrix = torch.tensor (t_df.to_numpy ())
        N_FEAT_TAGS += 1
        
        self.embed_dim     = embed_dim
        self.tag_embedding = nn.Embedding (N_FEAT_TAGS+1, embed_dim)
        self.tag_weights   = nn.Linear (N_FEAT_TAGS, 1)
        
        drop_prob          = 0.5
        self.ffn           = FFN (inputCount=(130+embed_dim), outputCount=0,
                                  hiddenLayerCounts=[(three_hidden_layers[0]+embed_dim),
                                                     (three_hidden_layers[1]+embed_dim),
                                                     (three_hidden_layers[2]+embed_dim)],
                                  drop_prob=drop_prob)
        self.outDense      = nn.Linear (three_hidden_layers[2]+embed_dim, 5)
        return
    
    def features2emb (self):
        
        all_tag_idxs = torch.LongTensor (np.arange (N_FEAT_TAGS))
        tag_bools    = self.features_tag_matrix
        f_emb        = self.tag_embedding (all_tag_idxs).repeat (130, 1, 1)
        f_emb        = f_emb * tag_bools[:, :, None]
        s            = torch.sum (tag_bools, dim=1)       
        f_emb        = torch.sum (f_emb, dim=-2) / s[:, None]
        return f_emb
    
    def forward (self, cat_featrs, features):
        
        cat_featrs = None
        features   = features.view (-1, N_FEATURES)
        f_emb      = self.features2emb ()
        features_2 = torch.matmul (features, f_emb)
        features   = torch.hstack ((features, features_2))
        x          = self.ffn (features)
        out_logits = self.outDense (x)
        return out_logits

In [ ]:
embNN_model = Emb_NN_Model ()

try:
    embNN_model.load_state_dict (torch.load ("../input/jane-embnn5-auc-400-400-400/Jane_EmbNN5_auc_400_400_400.pth"))
except:
    embNN_model.load_state_dict (torch.load ("../input/jane-embnn5-auc-400-400-400/Jane_EmbNN5_auc_400_400_400.pth", map_location='cpu'))
    
embNN_model = embNN_model.eval ()

# **Inference**

In [ ]:
import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
if True:

    for (test_df, pred_df) in tqdm(env_iter):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, feat_cols].values
            if np.isnan(x_tt.sum()):
                x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean

            cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
            cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
            feature_inp = np.concatenate((
                x_tt,
                np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
                np.array(cross_1_2).reshape(x_tt.shape[0], 1),
            ), axis=1)

            # torch_pred
            torch_pred = np.zeros((1, len(target_cols)))
            for model in model_list:
                torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
            torch_pred = np.median(torch_pred)
            
            # tf_pred
            tf_pred = np.median(np.mean([model(x_tt, training = False).numpy() for model in tf_models],axis=0))
            
            # torch embedding_NN pred
            x_tt    = torch.tensor (x_tt).float ().view (-1, 130)
            embnn_p = np.median (torch.sigmoid (embNN_model (None, x_tt)).detach ().cpu ().numpy ().reshape ((-1, 5)), axis=1)
            
            # avg
            pred_pr = torch_pred*0.4 + tf_pred*0.4 + embnn_p*0.2
            
            pred_df.action = np.where (pred_pr >= 0.5, 1, 0).astype (int)
        else:
            pred_df.action = 0
        env.predict(pred_df)